In [1]:
import pandas as pd
import numpy as np
from pgmpy.models import BayesianModel
from pgmpy.estimators import MaximumLikelihoodEstimator

In [2]:
# loading data
raw = pd.read_csv("forestfires.csv")
data = pd.DataFrame(raw)
data.head()

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0


In [3]:
# prepocessing data
data_binned = data.copy()
data_binned["FFMC"] = pd.cut(data["FFMC"], [0,20,40,60,80,100])
data_binned["DMC"] = pd.cut(data["DMC"], [0,100,200,300])
data_binned["ISI"] = pd.cut(data["ISI"], [0,20,40,60])
data_binned["temp"] = pd.cut(data["temp"], [0,20,40])
data_binned["RH"] = pd.cut(data["RH"], [0,20,40,60,80,100])
data_binned["wind"] = pd.cut(data["wind"], 5)
data_binned["rain"] = pd.cut(data["rain"], 5)
data_binned["area"] = pd.cut(data["area"], [-2,0,1100])
data_binned = data_binned.drop('DC', 1)
data_binned.head()

,X,Y,month,day,FFMC,DMC,ISI,temp,RH,wind,rain,area
0,7,5,mar,fri,"(80, 100]","(0, 100]","(0, 20]","(0, 20]","(40, 60]","(5.8, 7.6]","(-0.0064, 1.28]","(-2, 0]"
1,7,4,oct,tue,"(80, 100]","(0, 100]","(0, 20]","(0, 20]","(20, 40]","(0.391, 2.2]","(-0.0064, 1.28]","(-2, 0]"
2,7,4,oct,sat,"(80, 100]","(0, 100]","(0, 20]","(0, 20]","(20, 40]","(0.391, 2.2]","(-0.0064, 1.28]","(-2, 0]"
3,8,6,mar,fri,"(80, 100]","(0, 100]","(0, 20]","(0, 20]","(80, 100]","(2.2, 4.0]","(-0.0064, 1.28]","(-2, 0]"
4,8,6,mar,sun,"(80, 100]","(0, 100]","(0, 20]","(0, 20]","(80, 100]","(0.391, 2.2]","(-0.0064, 1.28]","(-2, 0]"


In [4]:
# split into train and test set
data_binned = data_binned.sample(frac=1) #shuffle
split = int(len(data_binned)/5)
data_binned_train = data_binned[:split]
data_binned_test = data_binned[split:]

In [5]:
# create graph for the model
G = BayesianModel()
G.add_nodes_from(data_binned)
G.add_edges_from([#("temp","area"),
                  #("wind","area"),
                  #("rain","area"),
                  ("RH","area"),
                  ("FFMC","area"),
                  ("DMC","area"),
                  ("ISI","area"),
                  ("RH","FFMC"),
                  ("wind","FFMC"),
                  ("rain","FFMC"),
                  ("temp","FFMC"),
                  ("RH","DMC"),
                  ("rain","DMC"),
                  ("temp","DMC"),
                  ("FFMC","ISI"),
                  ("wind","ISI"),
                  ("X","area"),
                  ("Y","area"),
                  ("day","RH"),
                  ("month","temp"),
                  ("month","wind")])

In [6]:
# fit model on data
G.fit(data_binned_train) # only used for discrete bayesnets?

Hier komt de code die het als LGBN runt zodat het continue variabelen kunnen zijn. Is trial and error dus no flame pls.

To instantiate an object of this class, one needs to provide a variable name, the value of the beta_0 term, the variance, a list of the parent variable names and a list of the coefficient values of the linear equation (beta_vector), where the list of parent variable names and beta_vector list is optional and defaults to None.

In [ ]:
from pgmpy.models import LinearGaussianBayesianNetwork
from pgmpy.factors.continuous import LinearGaussianCPD

In [ ]:
# prepocessing data
#make dummy variables from X,Y,Month,Day
data_with_dummies = pd.get_dummies(data, columns=["X","Y"], prefix=["X","Y"])
data_with_dummies = pd.get_dummies(data_with_dummies, columns=["month","day"])
data_with_dummies.head()

In [ ]:
#calculations here
#calculate variance of each column
variances = np.var(data,0)
print(variances)
#variances.get('X')

In [ ]:
model = LinearGaussianBayesianNetwork()

#Instantiation of data points
#cpd = LinearGaussianCPD(name, beta_0 term, variance, list_parent_names, list_coeff_val_lin_eq(beta vector)) last 2 are optional

model.add_nodes_from(data_with_dummies)
model.add_edges_from([("temp","area"),
                  ("wind","area"),
                  ("rain","area"),
                  ("RH","area"),
                  ("FFMC","area"),
                  ("DMC","area"),
                  ("ISI","area"),
                  ("RH","FFMC"),
                  ("wind","FFMC"),
                  ("rain","FFMC"),
                  ("temp","FFMC"),
                  ("RH","DMC"),
                  ("rain","DMC"),
                  ("temp","DMC"),
                  ("FFMC","ISI"),
                  ("wind","ISI")])
model.add_edges_from([(col,"area") for col in data_with_dummies if col.startswith('X')])
model.add_edges_from([(col,"rain") for col in data_with_dummies if col.startswith('Y')])
model.add_edges_from([(col,"RH") for col in data_with_dummies if col.startswith('day')])
model.add_edges_from([(col,"temp") for col in data_with_dummies if col.startswith('month')])
model.add_edges_from([(col,"wind") for col in data_with_dummies if col.startswith('month')])

estimator = MaximumLikelihoodEstimator(model,data_with_dummies)

#IMPORTANT: month should be replaced with the dummy variables of month, like below
#likewise for X, Y, day in cpd_area
#beta_0 = estimator.estimate_cpd('temp')
cpd_temp = LinearGaussianCPD('temp', estimator.estimate_cpd('temp').get_values(), variances.get('temp'), ['month'])
cpd_wind = LinearGaussianCPD('wind', estimator.estimate_cpd('wind'), variances.get('wind'), ['month'])
cpd_rain = LinearGaussianCPD('rain', estimator.estimate_cpd('rain'), variances.get('rain'), ['month'])
cpd_RH = LinearGaussianCPD('RH', estimator.estimate_cpd('RH'), variances.get('RH'), ['month'])
cpd_FFMC = LinearGaussianCPD('FFMC', estimator.estimate_cpd('FFMC'), variances.get('FFMC'), ['rain','RH','temp','wind'])
cpd_DMC = LinearGaussianCPD('DMC', estimator.estimate_cpd('DMC'), variances.get('DMC'), ['rain','RH','temp'])
cpd_ISI = LinearGaussianCPD('ISI', estimator.estimate_cpd('ISI'), variances.get('ISI'), ['FFMC','wind'])
cpd_area = LinearGaussianCPD('area', estimator.estimate_cpd('area'), variances.get('area'), ['rain','RH','temp','wind','month','FFMC','DMC','ISI','day','X','Y'])
#should every dummy var have a data point? think so but not sure, 44 in total (incl above)

#jgd = model.to_joint_gaussian()

#model.fit(data_with_dummies)


In [ ]:
tmp = estimator.estimate_cpd('temp').get_values()

In [ ]:
print(tmp)
print np.shape(tmp)

In [ ]:
[('x1', 'x2'), ('x2', 'x3')]
cpd1 = LinearGaussianCPD('x1', 1, 4)
cpd2 = LinearGaussianCPD('x2', -5, 4, ['x1'], [0.5])
cpd3 = LinearGaussianCPD('x3', 4, 3, ['x2'], [-1])
model.add_cpds(cpd1, cpd2, cpd3)
jgd = model.to_joint_gaussian()
jgd.variables

#structure score for eval 